In [12]:
import pandas as pd
import numpy as np

In [13]:
import yfinance as yf

ticker = "TCS.NS"

data = yf.download(ticker, start="2023-01-01", end="2023-12-31")

data.to_csv("TCS_2023.csv")

data.head()

/tmp/ipython-input-1514406877.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2023-01-01", end="2023-12-31")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS
Date,,,,,
2023-01-02,2986.108887,2991.053137,2962.074988,2985.696922,709547
2023-01-03,3031.796387,3039.716039,2971.322473,2977.456795,1245178
2023-01-04,3034.817383,3046.445390,3008.769260,3027.538590,1231668
2023-01-05,3031.567871,3055.555950,3004.878661,3049.055262,1826057
2023-01-06,2940.421875,3022.823819,2929.846914,3021.404630,2488376


In [14]:
df = pd.read_csv("TCS_2023.csv", skiprows=2)

df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
Date,,,,,
2023-01-02,2986.108887,2991.053137,2962.074988,2985.696922,709547
2023-01-03,3031.796387,3039.716039,2971.322473,2977.456795,1245178
2023-01-04,3034.817383,3046.445390,3008.769260,3027.538590,1231668
2023-01-05,3031.567871,3055.555950,3004.878661,3049.055262,1826057
2023-01-06,2940.421875,3022.823819,2929.846914,3021.404630,2488376


In [15]:
df = pd.read_csv("TCS_2023.csv", index_col=0, parse_dates=True, date_format = "%Y-%m-%d")

df.index.name = 'Date'

df = df[df['Close'] != 'TCS.NS']

df = df.apply(pd.to_numeric, errors='coerce')

df.dropna(inplace=True)

In [16]:
def add_sma(df, period=20):
  df['SMA'] = df['Close'].rolling(period).mean()
  return df

def add_rsi(df, period=14):
  delta = df['Close'].diff()
  gain = delta.clip(lower=0)
  loss = -delta.clip(upper=0)

  avg_gain = gain.rolling(period).mean()
  avg_loss = loss.rolling(period).mean()

  rs = avg_gain / avg_loss
  df['RSI'] = 100 - (100 / (1 + rs))
  return df

In [17]:
df = add_sma(df)
df = add_rsi(df)
df.dropna(inplace=True)

In [18]:
df['Signal'] = 0
df.loc[df['RSI'] < 40, 'Signal'] = 1

In [19]:
initial_capital = 100000
capital = initial_capital

in_trade = False
entry_price = 0
stop_loss = 0
take_profit = 0
shares = 0

trades = []
returns = []
portfolio_values = []

for i in range(len(df)):
  price = df['Close'].iloc[i]


  if not in_trade and df['Signal'].iloc[i] == 1:
    entry_price = price
    stop_loss = entry_price * (1 - 0.02)
    take_profit = entry_price * (1 + 0.05)
    shares = capital / entry_price
    in_trade =  True


  elif in_trade:

    if price <= stop_loss:
      exit_price = price
      profit = (exit_price - entry_price) * shares
      capital += profit
      trades.append(profit)
      returns.append(profit / (entry_price * shares))
      in_trade = False


    elif price >= take_profit:
      exit_price = price
      profit = (exit_price - entry_price) * shares
      capital += profit
      trades.append(profit)
      returns.append(profit / (entry_price * shares))
      in_trade = False

  portfolio_values.append(capital)


In [20]:
net_profit = capital - initial_capital
total_trades = len(trades)
winning_trades = len([t for t in trades if t > 0])
losing_trades = len([t for t in trades if t <= 0])

returns = np.array(returns)
sharpe_ratio = np.mean(returns) / np.std(returns) if len(returns) > 1 else 0

portfolio = np.array(portfolio_values)
cum_max = np.maximum.accumulate(portfolio)
drawdown = (portfolio - cum_max) / cum_max
max_drawdown = drawdown.min()

In [21]:
print("Net Profit:", net_profit)
print("Sharpe Ratio:", sharpe_ratio)
print("Maximum Drawdown:", max_drawdown)
print("Total Trades:", total_trades)
print("Winning Trades:", winning_trades)
print("Losing Trades:", losing_trades)

print("Winning + Losing =", winning_trades + losing_trades)

Net Profit: 18884.84532483628
Sharpe Ratio: 0.5152338428012716
Maximum Drawdown: -0.06307757233491226
Total Trades: 7
Winning Trades: 4
Losing Trades: 3
Winning + Losing = 7
